<a href="https://colab.research.google.com/github/pouyarahmti/Pytorch-Deep-Learning-Bootcamp/blob/main/01_pytorch_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Workflow


In [1]:
import torch
from torch import nn # nn contains all of PyTorch's building blocks for neural networks
import matplotlib.pyplot as plt


# Check PyTorch version
torch.__version__

'2.2.1+cu121'

## Data (preparing and loading)

data can be almost anything .... in Machine Learning.

* Excel Spreadsheet
* Images of any kind
* Videos
* Text
* Audio

Machine learning is a game of two parts:
1. Get data into the numerical format
2. Build a model to learn patterns in that numerical data


We'll use linear regression formula to make a straight line with *known* **parameters**.

In [8]:
# Create known parameters

weight = 0.7
bias = 0.3

# Create range values
start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [9]:
len(X), len(y)

(50, 50)

In [11]:
X.shape, y.shape

(torch.Size([50, 1]), torch.Size([50, 1]))

### Splitting data into training and test sets (one of most important machine learning workflow)

In [12]:
# Create a train/test split
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

In [13]:
len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)